In [1]:
import numpy as np

## Function 1 -  sigmoid function

*My solution*

In [2]:
def sigmoid(x):
    
    # the sigmoid function is 1/(1+e power -x)
    
    s = 1 / (1 + np.exp(-x))
    
    return s

In [3]:
# Try on scalar

x = np.array(10)
s = sigmoid(x)
s

0.9999546021312976

*Test on array*

In [4]:
x = np.array(np.array([1, 2, 3, 4, 5]))
s = sigmoid(x)
s

array([0.73105858, 0.88079708, 0.95257413, 0.98201379, 0.99330715])

**Testing it on their input from the unit test utils.py**

Code result approximately matches their output.

In [5]:
x = np.array([-0.46612273, -0.87671855, 0.54822123, -0.36443576, -0.87671855, 0.33688521
              , -0.87671855, 0.33688521, -0.36443576, -0.36443576, 0.54822123]).astype(float)

In [6]:
x

array([-0.46612273, -0.87671855,  0.54822123, -0.36443576, -0.87671855,
        0.33688521, -0.87671855,  0.33688521, -0.36443576, -0.36443576,
        0.54822123])

In [7]:
expected_s = np.array(
            [0.38553435, 0.29385824, 0.63372281, 0.40988622, 0.29385824, 0.5834337, 0.29385824, 0.5834337, 0.40988622,
             0.40988622, 0.63372281]).astype(float)

In [8]:
expected_s

array([0.38553435, 0.29385824, 0.63372281, 0.40988622, 0.29385824,
       0.5834337 , 0.29385824, 0.5834337 , 0.40988622, 0.40988622,
       0.63372281])

In [10]:
# testing with my code
observed_s = sigmoid(x)

In [11]:
observed_s

array([0.38553435, 0.29385824, 0.6337228 , 0.40988622, 0.29385824,
       0.58343371, 0.29385824, 0.58343371, 0.40988622, 0.40988622,
       0.6337228 ])

# Function 2 - Naive softmax and loss function

**Their input from the basic unit test is:**

In [12]:
center_word_vec = np.array([-0.27323645, 0.12538062, 0.95374082]).astype(float)

outside_word_idx = 3

outside_vectors = np.array([[-0.6831809, -0.04200519, 0.72904007],
                                    [0.18289107, 0.76098587, -0.62245591],
                                    [-0.61517874, 0.5147624, -0.59713884],
                                    [-0.33867074, -0.80966534, -0.47931635],
                                    [-0.52629529, -0.78190408, 0.33412466]]).astype(float)

*This shows the center word vector.*

In [13]:
center_word_vec

array([-0.27323645,  0.12538062,  0.95374082])

In [14]:
outside_word_idx

3

*This is basically the U matrix consisting of all outside word vectors. Each row should be an outside word vector.*

Here, its a 5x3 matrix, so there are 5 vectors and 3 dimensions.

In [15]:
outside_vectors

array([[-0.6831809 , -0.04200519,  0.72904007],
       [ 0.18289107,  0.76098587, -0.62245591],
       [-0.61517874,  0.5147624 , -0.59713884],
       [-0.33867074, -0.80966534, -0.47931635],
       [-0.52629529, -0.78190408,  0.33412466]])

In [16]:
outside_vectors.shape

(5, 3)

In [17]:
outside_vectors[outside_word_idx]

array([-0.33867074, -0.80966534, -0.47931635])

In [18]:
# for 1 pair, the numerator for calculating P(o|c) is exp(product of transposed outside vector and center vector)

outside_transposed = outside_vectors[outside_word_idx]

In [23]:
outside_vectors[outside_word_idx].T

array([-0.33867074, -0.80966534, -0.47931635])

In [24]:
outside_vectors[outside_word_idx].shape

(3,)

**This is the dot product of an outside word vector and center word vector, specifically the outside vector idx 3**

In [48]:
np.dot(outside_vectors[outside_word_idx],center_word_vec)

-0.46612272029364477

**However, no loops required as you can do a matrix multiplication of the whole outside word matrix U and 1 center word vector**

Here, -0.466122 is the dot product for the fourth index and is same as multiplying the respective vectors as shown above.

In [34]:
# Compare this with applying to entire matrix U

np.matmul(outside_vectors,center_word_vec)

array([ 0.87671856, -0.54822124, -0.3368852 , -0.46612272,  0.36443577])

**Their softmax function copied from utils.py**

In [35]:
def softmax(x):
    """Compute the softmax function for each row of the input x.
    It is crucial that this function is optimized for speed because
    it will be used frequently in later code. 

    Arguments:
    x -- A D dimensional vector or N x D dimensional numpy matrix.
    Return:
    x -- You are allowed to modify x in-place
    """
    orig_shape = x.shape

    if len(x.shape) > 1:
        # Matrix
        tmp = np.max(x, axis=1)
        x -= tmp.reshape((x.shape[0], 1))
        x = np.exp(x)
        tmp = np.sum(x, axis=1)
        x /= tmp.reshape((x.shape[0], 1))
    else:
        # Vector
        tmp = np.max(x)
        x -= tmp
        x = np.exp(x)
        tmp = np.sum(x)
        x /= tmp

    assert x.shape == orig_shape
    return x

**Test taking the softmax of above vector**

In [40]:
softmax(np.matmul(outside_vectors,center_word_vec))

array([0.41703564, 0.10030664, 0.12391154, 0.10888915, 0.24985703])

*Calculate the softmax by self to make sure using their function is correct.*

The self calculated vector == their softmax result.

In [49]:
np.exp(np.matmul(outside_vectors,center_word_vec))

array([2.40300145, 0.57797698, 0.7139908 , 0.62743028, 1.43970145])

In [50]:
0.62743028/(2.40300145+0.57797698+0.7139908+0.62743028+1.43970145)

0.10888915073782393

In [43]:
P = (np.exp(np.matmul(outside_vectors,center_word_vec)))/(np.sum(np.exp(np.matmul(outside_vectors,center_word_vec))))

In [46]:
P

array([0.41703564, 0.10030664, 0.12391154, 0.10888915, 0.24985703])

In [51]:
outside_vectors

array([[-0.6831809 , -0.04200519,  0.72904007],
       [ 0.18289107,  0.76098587, -0.62245591],
       [-0.61517874,  0.5147624 , -0.59713884],
       [-0.33867074, -0.80966534, -0.47931635],
       [-0.52629529, -0.78190408,  0.33412466]])

In [70]:
# for calculating gradient wrt center word, reshaping this is necessary
(0.41703564 * outside_vectors[0])

array([-0.28491078, -0.01751766,  0.30403569])

In [69]:
P[:,np.newaxis] * outside_vectors

array([[-0.28491078, -0.01751766,  0.30403569],
       [ 0.01834519,  0.07633193, -0.06243646],
       [-0.07622774,  0.063785  , -0.07399239],
       [-0.03687757, -0.08816377, -0.05219235],
       [-0.13149858, -0.19536423,  0.0834834 ]])

In [71]:
np.sum(P[:,np.newaxis] * outside_vectors,axis=0)

array([-0.51116949, -0.16092873,  0.19889788])

In [74]:
-outside_vectors[outside_word_idx] + np.sum(P[:,np.newaxis] * outside_vectors,axis=0)

array([-0.17249875,  0.64873661,  0.67821423])

**Gradient of loss wrt all outer words U**

In [75]:
outside_vectors

array([[-0.6831809 , -0.04200519,  0.72904007],
       [ 0.18289107,  0.76098587, -0.62245591],
       [-0.61517874,  0.5147624 , -0.59713884],
       [-0.33867074, -0.80966534, -0.47931635],
       [-0.52629529, -0.78190408,  0.33412466]])

In [77]:
all_outer_probs = naive_softmax_loss_and_gradient(center_word_vec,outside_word_idx,outside_vectors)

In [78]:
all_outer_probs

array([0.41703564, 0.10030664, 0.12391154, 0.10888915, 0.24985703])

In [80]:
all_outer_probs.shape

(5,)

In [81]:
center_word_vec.shape

(3,)

In [82]:
all_outer_probs

array([0.41703564, 0.10030664, 0.12391154, 0.10888915, 0.24985703])

In [92]:
grad_outside_vecs = all_outer_probs[:,np.newaxis] * center_word_vec

In [93]:
grad_outside_vecs

array([[-0.11394934,  0.05228819,  0.39774391],
       [-0.02740743,  0.01257651,  0.09566654],
       [-0.03385715,  0.01553611,  0.11817949],
       [-0.02975249,  0.01365259,  0.10385203],
       [-0.06827005,  0.03132723,  0.23829885]])

In [ ]:
# here, also subtract the center word vector from outer word of interest
 = 

In [94]:
all_outer_probs[0]

0.4170356381889728

In [100]:
(all_outer_probs[outside_word_idx] * center_word_vec) - center_word_vec

array([ 0.24348396, -0.11172803, -0.84988879])

In [101]:
def naive_softmax_loss_and_gradient(center_word_vec,outside_word_idx,outside_vectors):
    
    # Get the dot product of outer word vector and center word vector for each outer word
    # This will generate a 1 dimensional vector with n elements (n,)
    # Each element represents u.v for outer word+center word pairs
    dot_products = np.matmul(outside_vectors,center_word_vec)
    
    # Apply softmax to this using their util function which will calculate the softmax for each outer word-center word pair
    # Ultimately, this will give the probabilities for each outer word to be around context word
    all_outer_probs = softmax(dot_products)
    
    # Up until this point, we have derived the P(o|c) for each outer word-center word pair
    # Next, to calculate loss we need to take the negative log of each probability
    # Get the probability for the outer word index of interest
    outer_word_prob = all_outer_probs[outside_word_idx]
    
    # get the loss for this outer word and center word pair
    # J = -log P(o|c)
    loss = -np.log(outer_word_prob)
    
    # the gradient of J wrt center word vector vc:
    # dJ/dv_c = -u_o + Sum(predicted probabilities of each outside word x respective word embedding)
    # all_outer_probs[:,np.newaxis] means that the 1d array is converted to a 2D array
    # This conversion is imp. since we want an element-wise operation of each predicted prob*embedding for that word
    grad_center_vec = -outside_vectors[outside_word_idx] + np.sum(all_outer_probs[:,np.newaxis] * outside_vectors,axis=0)
    
    # gradient of J wrt outside word matrix U
    # dJ/dU = (y_predicted * center word vector) - center word vector for the true outside word
    # Assuming that the true word in this case is the outside word at index outside_word_idx
    # dJ/dU = (y_predicted * center word vector) for all other outside words
    grad_outside_vecs = (all_outer_probs[:,np.newaxis] * center_word_vec)
    
    # Only for the outside word of interest, subtract the center word vector
    grad_outside_vecs[outside_word_idx] = grad_outside_vecs[outside_word_idx] - center_word_vec
    
    return all_outer_probs